In [628]:
import pandas as pd
import numpy as np
import os


In [629]:
print(os.getcwd())
os.chdir('../raw_data')
print(os.getcwd())

/Users/kumo/hec/courses/data_challenge_uw/Data-Challenge-URW-HEC/raw_data
/Users/kumo/hec/courses/data_challenge_uw/Data-Challenge-URW-HEC/raw_data


In [630]:
data = pd.read_csv('cleaned_data.csv')
data.head(5)

,Unnamed: 0,external_mall_id,block_id,average_dwell_time,median_dwell_time,store_id,store_visits_monthly_average,store_window_flow_monthly_average,store_visits_daily_average,store_window_flow_daily_average,...,store_branch_level_1,store_branch_level_2,store_branch_level_3,store_sales_r12m,store_rent_r12m,store_area,service_charges_r12m,marketing_costs_r12m,real_estate_taxes_r12m,maintenance_works_r12m
0,0,18,11273,249.520051,180.0,9257.0,9810.25,472952.50,509.62,24568.96,...,Jewellery,Jewellery & Watches,Modern Jewellery & Watches,886377.00,NaN,104.0,25034.0,0.0,11957.0,6337.0
1,1,18,11271,190.005359,120.0,9443.0,4891.25,362107.00,254.09,18810.75,...,Fashion apparel,Family Fashion,Family Fashion,20883319.00,NaN,203.0,33465.0,0.0,11957.0,6337.0
2,2,17,4319,1325.216860,780.0,5697.0,1981.23,280719.08,70.37,9970.90,...,Services,Imaterial services,Travel Agency,2339819.57,140492.0,72.0,18000.0,980.0,11957.0,6337.0
3,3,36,1201,621.422549,240.0,7179.0,29753.50,751216.75,1226.95,30978.01,...,Food & Beverage Services,Sweet moments,Confectionery,1129538.00,89064.0,107.0,80035.0,5038.0,11957.0,24846.0
4,4,36,1212,217.833860,120.0,7288.0,6082.50,357884.00,253.44,14911.83,...,Fashion apparel,Men's Fashion,Men's Fashion,340091.00,2181010.0,208.0,69226.0,0.0,11957.0,6337.0


In [631]:
data.columns

Index(['Unnamed: 0', 'external_mall_id', 'block_id', 'average_dwell_time',
       'median_dwell_time', 'store_id', 'store_visits_monthly_average',
       'store_window_flow_monthly_average', 'store_visits_daily_average',
       'store_window_flow_daily_average', 'store_total_visits',
       'store_total_window_flow', 'days_with_traffic_data', 'mall_name',
       'mall_country', 'mall_id', 'total_mall_area', 'store_name',
       'store_year_opened', 'store_year_closed', 'store_branch_level_1',
       'store_branch_level_2', 'store_branch_level_3', 'store_sales_r12m',
       'store_rent_r12m', 'store_area', 'service_charges_r12m',
       'marketing_costs_r12m', 'real_estate_taxes_r12m',
       'maintenance_works_r12m'],
      dtype='object')

# further cleaning

## cost

In [632]:
cost_cols = ['service_charges_r12m', 'marketing_costs_r12m', 'real_estate_taxes_r12m', 'maintenance_works_r12m']
data[cost_cols].min(axis=0)
# negative values should not exist, we determine to use 0 to replace them.

service_charges_r12m     -86042.0
marketing_costs_r12m     -41943.0
real_estate_taxes_r12m   -16695.0
maintenance_works_r12m        0.0
dtype: float64

In [633]:
print(f"percentage(%) of negative values\n{((data[cost_cols]<0).sum(axis=0)/data.shape[0])*100}")

percentage(%) of negative values
service_charges_r12m      1.673066
marketing_costs_r12m      0.246846
real_estate_taxes_r12m    0.054855
maintenance_works_r12m    0.000000
dtype: float64


it's okay to directly drop them, not many

In [634]:
print(f"to dorp {((data[cost_cols] < 0).any(axis=1).sum()/data.shape[0]) * 100}%")

to dorp 1.9747668678003292%


In [635]:
data = data.loc[~(data[cost_cols] < 0).any(axis=1)]

In [636]:
# replace them with zero
data[cost_cols] = data[cost_cols].applymap(lambda x:max(x, 0))
data['total_costs_r12m'] = data[cost_cols].sum(axis=1)

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/2867994817.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[cost_cols] = data[cost_cols].applymap(lambda x:max(x, 0))


## visitor

In [637]:
data['store_visits_monthly_average'].describe()

count      3574.000000
mean      21500.184063
std       33539.946814
min           0.000000
25%        4745.547500
50%       10213.460000
75%       23601.375000
max      444736.540000
Name: store_visits_monthly_average, dtype: float64

In [638]:
(data['store_visits_monthly_average'] == 0).sum()
# directly drop

np.int64(5)

there're zero observations

In [639]:
data.shape

(3574, 31)

In [640]:
data.drop(data[data['store_visits_monthly_average']==0].index).shape

(3569, 31)

In [641]:
data = data.drop(data[data['store_visits_monthly_average']==0].index)

## area

In [642]:
data['store_area'].min()
# area is 0, that's no way

np.float64(0.0)

In [643]:
(data['store_area'] == 0).sum()/data.shape[0]
# 10 percent
# too much, in case of generating outliers, using the median value to fill it

np.float64(0.09106192210703279)

In [644]:
data['store_area'].median()

np.float64(160.0)

In [645]:
data.loc[data['store_area']!=0, 'store_area'].sort_values().head(5)

634      7.3
1004    14.0
926     14.2
207     15.0
3166    15.0
Name: store_area, dtype: float64

In [646]:
data['store_area'] = data['store_area'].apply(lambda x: 160 if x<=0 else x)

## sales value

In [647]:
(data['store_sales_r12m']==0).sum()

np.int64(4)

In [648]:
data = data.loc[data['store_sales_r12m']!=0]

# metrics

In [649]:
identity = ['store_id', 'block_id', 'mall_id', 'external_mall_id']
metrics = data[identity]

make them based on month

## traffic

### traffic cost efficiency

In [650]:
# traffic cost efficiency
metrics['tfc_cst_efc'] = data['store_window_flow_monthly_average']/(data['total_costs_r12m']/12) 

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/2077493001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['tfc_cst_efc'] = data['store_window_flow_monthly_average']/(data['total_costs_r12m']/12)


### visit rate

In [651]:
# visit rate
# there are monthly, daily and total 
metrics['monthly_visit/window'] = data['store_visits_monthly_average'] / data['store_window_flow_monthly_average']
metrics['daily_visit/window'] = data['store_visits_daily_average'] / data['store_window_flow_daily_average']
metrics['total_visit/window'] = data['store_total_visits'] / data['store_total_window_flow']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1795006489.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['monthly_visit/window'] = data['store_visits_monthly_average'] / data['store_window_flow_monthly_average']
/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1795006489.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['daily_visit/window'] = data['store_visits_daily_average'] / data['store_window_flow_daily_average']
/var/folders/z9/zf5v62yj5_75v0dw

In [652]:
metrics.head(10)
# since they are almost the same, we only keep one
metrics.drop(['daily_visit/window', 'total_visit/window'], axis=1, inplace=True)
metrics.rename(columns={'monthly_visit/window':'visit_rate'}, inplace=True)

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1917769999.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics.drop(['daily_visit/window', 'total_visit/window'], axis=1, inplace=True)
/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1917769999.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics.rename(columns={'monthly_visit/window':'visit_rate'}, inplace=True)


## Sales

### revenue per visitor

In [653]:
# revenue per visit
# time unit, month
metrics['RPV'] = (data['store_sales_r12m']/12)/data['store_visits_monthly_average']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/2476251328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['RPV'] = (data['store_sales_r12m']/12)/data['store_visits_monthly_average']


### revenue per m2

In [654]:
# revenue per m2
# RPM 
metrics['RPM'] = (data['store_sales_r12m']/12)/data['store_area']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/3968750639.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['RPM'] = (data['store_sales_r12m']/12)/data['store_area']


### revenue efficiency

In [655]:
# Rvenue efficiency
metrics['RE'] = data['store_sales_r12m']/data['total_costs_r12m']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/3601272623.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['RE'] = data['store_sales_r12m']/data['total_costs_r12m']


In [656]:
metrics['RE'].sort_values()

2174    0.001609
1657    0.012399
2923    0.032193
1534    0.061527
1263    0.071061
          ...   
3628         NaN
3639         NaN
3640         NaN
3641         NaN
3643         NaN
Name: RE, Length: 3565, dtype: float64

In [657]:
data.columns

Index(['Unnamed: 0', 'external_mall_id', 'block_id', 'average_dwell_time',
       'median_dwell_time', 'store_id', 'store_visits_monthly_average',
       'store_window_flow_monthly_average', 'store_visits_daily_average',
       'store_window_flow_daily_average', 'store_total_visits',
       'store_total_window_flow', 'days_with_traffic_data', 'mall_name',
       'mall_country', 'mall_id', 'total_mall_area', 'store_name',
       'store_year_opened', 'store_year_closed', 'store_branch_level_1',
       'store_branch_level_2', 'store_branch_level_3', 'store_sales_r12m',
       'store_rent_r12m', 'store_area', 'service_charges_r12m',
       'marketing_costs_r12m', 'real_estate_taxes_r12m',
       'maintenance_works_r12m', 'total_costs_r12m'],
      dtype='object')

## cost

### cost per m2

In [658]:
# cost per m2
metrics['CPM'] = (data['total_costs_r12m']/12)/data['store_area']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/2759233445.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['CPM'] = (data['total_costs_r12m']/12)/data['store_area']


### profit margin

In [659]:
# profit margin 
metrics['PM'] = (data['store_sales_r12m']-data['total_costs_r12m'])/data['store_sales_r12m']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1555608295.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['PM'] = (data['store_sales_r12m']-data['total_costs_r12m'])/data['store_sales_r12m']


## engagement

In [660]:
# using median dwell in case outlier effect
metrics['Space_u'] = data['median_dwell_time']/data['store_area']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/2825203873.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['Space_u'] = data['median_dwell_time']/data['store_area']


In [661]:
metrics.head(5)

,store_id,block_id,mall_id,external_mall_id,tfc_cst_efc,visit_rate,RPV,RPM,RE,CPM,PM,Space_u
0,9257.0,11273,54.0,18,130.987583,0.020743,7.529344,710.237981,20.457372,34.717949,0.951118,1.730769
1,9443.0,11271,54.0,18,83.952240,0.013508,355.793833,8572.791051,403.472227,21.247537,0.997522,0.591133
2,5697.0,4319,15.0,17,90.374764,0.007058,98.416117,2708.124502,62.773504,43.141204,0.984070,10.833333
3,7179.0,1201,35.0,36,73.965350,0.039607,3.163600,879.702492,9.267928,94.919003,0.892101,2.242991
4,7288.0,1212,35.0,36,49.070018,0.016996,4.659419,136.254407,3.885866,35.064103,0.742657,0.576923


In [662]:
metrics.isna().sum(axis=0)

store_id              0
block_id              0
mall_id               0
external_mall_id      0
tfc_cst_efc           0
visit_rate            0
RPV                 869
RPM                 869
RE                  869
CPM                   0
PM                  869
Space_u               0
dtype: int64

# comprehensive metric

In [663]:
metrics.columns

Index(['store_id', 'block_id', 'mall_id', 'external_mall_id', 'tfc_cst_efc',
       'visit_rate', 'RPV', 'RPM', 'RE', 'CPM', 'PM', 'Space_u'],
      dtype='object')

In [664]:
m_cols = ['tfc_cst_efc','visit_rate', 'RPV', 'RPM', 'RE', 'CPM', 'PM', 'Space_u']
metrics[m_cols].describe()

,tfc_cst_efc,visit_rate,RPV,RPM,RE,CPM,PM,Space_u
count,3565.000000,3565.000000,2696.000000,2696.000000,2696.000000,3565.000000,2696.000000,3565.000000
mean,183.556293,0.038342,72.617138,3131.268222,133.297128,56.539222,0.506107,3.419174
std,216.508886,0.055595,334.382879,11163.078715,536.926978,159.972278,12.107151,6.137240
min,0.582914,0.000338,0.003685,0.396387,0.001609,0.136030,-620.374000,0.023414
25%,61.475196,0.009581,2.893680,169.087922,7.917044,10.961979,0.873689,0.782269
50%,122.875103,0.019243,9.566621,535.739046,22.482203,23.452070,0.955520,1.512605
75%,232.253684,0.042927,32.011828,1683.737958,66.371001,52.822125,0.984933,3.428571
max,4484.618035,0.790122,9634.972150,169864.125514,9772.530218,4489.553824,0.999898,104.000000


In [665]:
# obviously there's null values, especiall in revenue

In [666]:
metrics.isna().sum(axis=0)

store_id              0
block_id              0
mall_id               0
external_mall_id      0
tfc_cst_efc           0
visit_rate            0
RPV                 869
RPM                 869
RE                  869
CPM                   0
PM                  869
Space_u               0
dtype: int64

In [667]:
data.isna().sum(axis=0)[data.isna().sum(axis=0)!=0]
# obviously the null values come from sales

store_sales_r12m    869
store_rent_r12m     965
dtype: int64

In [668]:
# better not using null values
metrics = metrics.loc[~metrics.isna().any(axis=1)]
metrics.reset_index(drop=True, inplace=True)

1. drop  
2. use the median value to replace them

## AHP(Analytic hierarchy process)
refer to [AHP](https://en.wikipedia.org/wiki/Analytic_hierarchy_process)

the AHP Comparison Scale

AHP uses a standardized scale (1–9) for expressing relative importance. The common scale looks like this:

| Numeric Value | Verbal Judgment               | Interpretation                                               |
|---------------|-------------------------------|--------------------------------------------------------------|
| 1             | Equal importance             | Criterion \(i\) is equally important as criterion \(j\).     |
| 3             | Moderate importance          | Criterion \(i\) is moderately more important than \(j\).     |
| 5             | Strong importance            | Criterion \(i\) is strongly more important than \(j\).       |
| 7             | Very strong importance       | Criterion \(i\) is much more important than \(j\).           |
| 9             | Extreme importance           | Criterion \(i\) is extremely more important than \(j\).      |
| 2, 4, 6, 8    | Intermediate values          | Compromise or midpoints between the above judgments.         |


build the APH matrix initially. 
Assumption:   
1. the metrix under the same aspect are of the same importance
2. importance: sales > engagement >= traffic > cost (underlying idea: taking retail's profota) 
3. will intially obtain aspect metric through averaging sub_metrix

Priority Ranking:
1. Sales Performance: Revenue directly reflects the store's contribution to the mall's financial success.
2. Traffic: High traffic ensures sustained visibility and opportunities for conversions across the mall.
3. Engagement: It reflects user experience, contributing to the success of mall's entertainment attribute
4. Cost: While important for profitability, it’s secondary to attracting and retaining high-performing tenants.

### AHP matrix

In [669]:
sales = ["sales_rpv", "sales_rpm", "sales_re", "sales_pm"]
engagement = ['engagement']
traffic = ["traffic_tce", "traffic_vr"]
cost = ["cost_cpm"]
criteria = sales + engagement + traffic + cost


In [670]:
metrics.rename(columns={'tfc_cst_efc':'traffic_tce', 'visit_rate':'traffic_vr', 'RPV':'sales_rpv', 'RPM':'sales_rpm', 
                        'RE':'sales_re', 'CPM':'cost_cpm', 'PM':'sales_pm', 'Space_u':'engagement'}, inplace=True)

In [671]:
# Pairwise comparison matrix (AHP)
M = np.array([
    [1,     2,    2.5,    4],  # sales vs others
    [1/2,   1,    2,    3],  # engagement vs others
    [2/5,   1/2,  1,    2],  # traffic vs others
    [1/4,   1/3,  1/2,  1]   # cost vs others
], dtype=float)

### winsorization and normalization

In [672]:
metrics[criteria].describe()

,sales_rpv,sales_rpm,sales_re,sales_pm,engagement,traffic_tce,traffic_vr,cost_cpm
count,2696.000000,2696.000000,2696.000000,2696.000000,2696.000000,2696.000000,2696.000000,2696.000000
mean,72.617138,3131.268222,133.297128,0.506107,3.331107,181.970848,0.039114,57.579761
std,334.382879,11163.078715,536.926978,12.107151,5.815162,224.563500,0.056776,174.939475
min,0.003685,0.396387,0.001609,-620.374000,0.023414,0.582914,0.000338,0.140316
25%,2.893680,169.087922,7.917044,0.873689,0.779221,59.774213,0.009899,11.158853
50%,9.566621,535.739046,22.482203,0.955520,1.500000,121.512100,0.019841,23.467242
75%,32.011828,1683.737958,66.371001,0.984933,3.428571,227.770461,0.044603,52.583586
max,9634.972150,169864.125514,9772.530218,0.999898,76.842105,4484.618035,0.790122,4489.553824


In [673]:
metrics['sales_rpm'].sort_values()

1593         0.396387
1210         0.461610
1097         0.544265
148          0.845121
2260         1.052957
            ...      
1225    127173.068667
2441    143010.390411
1424    150536.274242
71      151808.773148
343     169864.125514
Name: sales_rpm, Length: 2696, dtype: float64

In [674]:
from scipy.stats.mstats import winsorize

In [675]:
# Define percentile limits for winsorization
lower_limit = 0.01
upper_limit = 0.99

# Calculate global thresholds for each column
thresholds = {}
for col in criteria:
    thresholds[col] = {
        "lower": np.percentile(metrics[col], lower_limit * 100),
        "upper": np.percentile(metrics[col], upper_limit * 100)
    }

In [676]:
def is_within_threshold(row, thresholds):
    for col, thresh in thresholds.items():
        if row[col] < thresh["lower"] or row[col] > thresh["upper"]:
            return False
    return True


In [677]:
metrics_filtered = metrics[metrics.apply(lambda row: is_within_threshold(row, thresholds), axis=1)]

In [678]:
metrics_filtered.shape[0]/metrics.shape[0]
# 10 are droped

0.9013353115727003

In [679]:
metrics_filtered.describe()

,store_id,block_id,mall_id,external_mall_id,traffic_tce,traffic_vr,sales_rpv,sales_rpm,sales_re,cost_cpm,sales_pm,engagement
count,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000
mean,5831.421811,5528.364609,27.506173,20.720988,168.389713,0.034991,41.168393,1996.657443,80.149112,44.389301,0.855346,2.875594
std,3921.499627,2897.273531,24.294961,8.141979,156.855708,0.040855,101.382666,4596.122316,185.041039,61.590473,0.329050,3.694524
min,1.000000,2.000000,0.000000,7.000000,5.985696,0.001515,0.084213,4.676029,0.275361,0.701492,-2.631599,0.135389
25%,1909.000000,4308.000000,5.000000,14.000000,61.648912,0.010237,3.025765,182.425672,8.429985,11.727604,0.881376,0.798670
50%,5426.500000,5983.500000,17.000000,19.000000,121.512100,0.019862,9.433694,534.609825,22.405043,23.564513,0.955367,1.500000
75%,9912.750000,7083.750000,57.000000,28.000000,222.699440,0.042557,30.168256,1564.259883,61.249625,50.281001,0.983673,3.243243
max,12427.000000,13815.000000,68.000000,36.000000,991.128297,0.266338,1027.423705,51511.628915,2045.211364,526.696759,0.999511,27.115385


In [680]:
# much better
# do normalization
from sklearn.preprocessing import MinMaxScaler, StandardScaler
min_max_scaler = MinMaxScaler()
metrics_filtered[criteria] = min_max_scaler.fit_transform(metrics_filtered[criteria])

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1666876784.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics_filtered[criteria] = min_max_scaler.fit_transform(metrics_filtered[criteria])


In [681]:
metrics_filtered[criteria].describe()

,sales_rpv,sales_rpm,sales_re,sales_pm,engagement,traffic_tce,traffic_vr,cost_cpm
count,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000
mean,0.039991,0.038674,0.039059,0.960297,0.101564,0.164853,0.126409,0.083057
std,0.098685,0.089233,0.090487,0.090620,0.136936,0.159221,0.154274,0.117093
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.002863,0.003451,0.003988,0.967466,0.024584,0.056503,0.032935,0.020962
50%,0.009101,0.010289,0.010822,0.987843,0.050579,0.117269,0.069281,0.043466
75%,0.029283,0.030279,0.029817,0.995638,0.115191,0.219982,0.154979,0.094258
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [682]:
# generate metrics for each aspect
metrics_filtered['sales'] = metrics_filtered[sales].mean(axis=1)
metrics_filtered['traffic'] = metrics_filtered[traffic].mean(axis=1)
metrics_filtered['cost'] = metrics_filtered[cost].mean(axis=1)

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/470875187.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics_filtered['sales'] = metrics_filtered[sales].mean(axis=1)
/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/470875187.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics_filtered['traffic'] = metrics_filtered[traffic].mean(axis=1)
/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/470875187.py:4: SettingWithCopyWarning: 
A value is trying 

In [683]:
metrics_filtered = metrics_filtered[['store_id', 'block_id', 'mall_id', 'external_mall_id', 'sales', 'engagement', 'traffic', 'cost']]

In [684]:
metrics_filtered

,store_id,block_id,mall_id,external_mall_id,sales,engagement,traffic,cost
0,9257.0,11273,54.0,18,0.254372,0.059132,0.099746,0.064671
1,9443.0,11271,54.0,18,0.427303,0.016892,0.062214,0.039061
2,5697.0,4319,15.0,17,0.293628,0.396514,0.053296,0.080685
3,7179.0,1201,35.0,36,0.248701,0.078117,0.106422,0.179122
4,7288.0,1212,35.0,36,0.234509,0.016365,0.051095,0.065329
...,...,...,...,...,...,...,...,...
2691,10065.0,6009,57.0,23,0.255821,0.014153,0.148827,0.013136
2692,11292.0,6514,64.0,26,0.239921,0.030564,0.142561,0.046544
2693,1171.0,4706,4.0,10,0.271224,0.194880,0.048717,0.218667
2694,1391.0,4535,4.0,10,0.199458,0.022313,0.383761,0.009283


### calculate the weights

In [685]:
def ahp_weights(pairwise_matrix):
    """
    Given a pairwise comparison matrix, returns (weights, CR).
    weights: 1D numpy array of derived weights
    CR: consistency ratio
    """
    # 1. Normalize columns
    col_sum = pairwise_matrix.sum(axis=0)
    normalized = pairwise_matrix / col_sum
    
    # 2. Average each row -> priority vector
    weights = normalized.mean(axis=1)
    
    # 3. Consistency check
    # Compute lambda_max
    n = pairwise_matrix.shape[0]
    lamda_max = (col_sum * weights).sum()
    
    # Consistency Index (CI)
    CI = (lamda_max - n) / (n - 1)
    
    # Random Index (RI) for n=4 (AHP standard table) is about 0.90
    # (You can adjust if you have a more precise reference table)
    RI = 0.90
    
    CR = CI / RI
    
    return weights, CR

weights, CR = ahp_weights(M)

In [686]:
criteria = ['sales',
       'engagement', 'traffic', 'cost']

In [687]:
print("Derived AHP weights for each criterion:")
for c, w in zip(criteria, weights):
    print(f"  {c}: {w:.3f}")

print(f"\nAHP Consistency Ratio (CR): {CR:.3f}")


Derived AHP weights for each criterion:
  sales: 0.451
  engagement: 0.282
  traffic: 0.171
  cost: 0.097

AHP Consistency Ratio (CR): 0.015


since the CR is smaller than 0.1, it's considered consistent

### topsis

In [688]:
def topsis(df, criteria, weights, beneficial):
    """
    Perform TOPSIS given:
    - df: DataFrame with columns = criteria, rows = stores
    - criteria: list of criterion names in the same order as 'weights' and 'beneficial'
    - weights: AHP-derived weights (1D array or list)
    - beneficial: list of booleans; True if criterion is beneficial, False if cost-like
    
    Returns: DataFrame with the final closeness scores and ranks.
    """
    # 1. Extract numerical data as matrix
    data_matrix = df[criteria].values.astype(float)
    n_stores, n_criteria = data_matrix.shape
    
    # 2. Normalize columns
    # denominator for each column = sqrt of sum of squares
    denom = np.sqrt((data_matrix ** 2).sum(axis=0))
    norm_matrix = data_matrix / denom
    
    # 3. Multiply by weights
    weighted_matrix = norm_matrix * weights
    
    # 4. Determine ideal (best) and negative-ideal (worst) for each criterion
    ideal_solution = np.zeros(n_criteria)
    neg_ideal_solution = np.zeros(n_criteria)
    
    for j in range(n_criteria):
        if beneficial[j]:
            ideal_solution[j] = weighted_matrix[:, j].max()
            neg_ideal_solution[j] = weighted_matrix[:, j].min()
        else:
            # For non-beneficial (cost), ideal is min, negative-ideal is max
            ideal_solution[j] = weighted_matrix[:, j].min()
            neg_ideal_solution[j] = weighted_matrix[:, j].max()
    
    # 5. Calculate distances
    dist_to_ideal = np.sqrt(((weighted_matrix - ideal_solution) ** 2).sum(axis=1))
    dist_to_neg_ideal = np.sqrt(((weighted_matrix - neg_ideal_solution) ** 2).sum(axis=1))
    
    # 6. Calculate relative closeness
    closeness = dist_to_neg_ideal / (dist_to_ideal + dist_to_neg_ideal)
    
    # 7. Create result DataFrame
    result_df = df.copy()
    result_df["TOPSIS_Closeness"] = closeness
    # Higher closeness -> better rank
    result_df["Rank"] = result_df["TOPSIS_Closeness"].rank(ascending=False, method='dense').astype(int)
    
    return result_df.sort_values("TOPSIS_Closeness", ascending=False)

### obtain the rank

In [690]:
# Define beneficial vs. non-beneficial
# True = beneficial, False = non-beneficial
beneficial_criteria = [True, True, False, True]  # Traffic, Sales, Cost, Engagement

# Calculate final ranking
results = topsis(metrics_filtered, criteria, weights, beneficial_criteria)

# Print the final results
results


,store_id,block_id,mall_id,external_mall_id,sales,engagement,traffic,cost,TOPSIS_Closeness,Rank
66,4384.0,6677,26.0,22,0.622756,0.671812,0.043457,0.243480,0.664139,1
1345,511.0,7098,1.0,19,0.289161,1.000000,0.040449,0.247065,0.642234,2
1155,12117.0,11480,68.0,32,0.257096,0.966826,0.013247,0.208264,0.621835,3
973,11324.0,6449,64.0,26,0.309662,0.886701,0.033031,0.158803,0.616156,4
2439,5779.0,4403,15.0,17,0.494184,0.717740,0.082490,0.034596,0.611865,5
...,...,...,...,...,...,...,...,...,...,...
1533,2207.0,6885,5.0,33,0.238697,0.005513,0.490538,0.004007,0.167975,2426
943,12109.0,11487,68.0,32,0.255709,0.000906,0.602363,0.000305,0.166443,2427
2038,4760.0,5066,17.0,14,0.244244,0.003234,0.537428,0.000549,0.165224,2428
683,4667.0,2834,19.0,12,0.234331,0.011647,0.522639,0.002414,0.162098,2429
